In [10]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cts
import pylcp
import time
import random
from tqdm import tqdm
from scipy.interpolate import interp1d,interp2d,RegularGridInterpolator
from scipy import stats
from scipy import integrate
from celluloid import Camera
from bayes_opt import BayesianOptimization
import json
import seaborn as sns
import White_class
from scipy import special
import pymongo
import Expected_model_simulation
from pymongo import MongoClient
from scipy.optimize import root_scalar


In [2]:
#Main variables

main_detune = 17
sideband_detune = 120
white_detune = 10
laser_on = 200000
laser_off = 400015
MOT_power = 50
v0_start=1
v0_step=1
v0_end=25
t0_start=0
t0_step=1
t0_end=1000000


Gamma = 22 # Hz to kHz, Decay rate
wavelength = 359.3e-9 # m to mm
k = 1/wavelength*2*np.pi #x_0
x0 = 1/k
t0 = 1/Gamma*1/(2*np.pi*1e6)
v0 = x0/t0
m0 = cts.hbar*t0/x0**2
a0 = x0/t0**2
F0 = cts.hbar/(x0*t0)
mass = 43*cts.value('atomic mass constant')/m0 # m_0
mag_field_grad = 1252.8168984164048*x0
waist = 0.012/x0
v_max = 25.
z_max = 384.855e-3/x0
z_start = 384.855e-3/x0
omega = 2*np.pi*(cts.c/wavelength) #Transition frequency
Isat = np.pi*cts.h*cts.c*Gamma*2*np.pi*1e6/3*1/(wavelength)**3
t_eval = np.arange(t0_start,t0_end,t0_step)

# The detunings used in the PRAs:
intensities = 2.*MOT_power*1e-3/(np.pi*0.012**2)/Isat

#Define the hamiltonian
H0_X, Bq_X, U_X, Xbasis = pylcp.hamiltonians.XFmolecules.Xstate(N=1,I=0.5,B=15496.8125/Gamma,
gamma = 50.697/Gamma,b=154.7/Gamma, c=178.5/Gamma,gI=5.585,gS=2.0023193043622,
    muB = cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,return_basis=True
    )

# b : SI coupling(isotropic), c : Iz Sz coupling(anisotropic), cc : I N coupling, gamma : S N coupling

E_X = np.unique(np.diag(H0_X))

H0_A, Bq_A, Abasis = pylcp.hamiltonians.XFmolecules.Astate(J=0.5,I=0.5,
    P=+1,B=15788.2/Gamma,D=0.,H=0.,a=0./Gamma,b=-0.4/Gamma,c=0.,q=0., p=15./Gamma,
    muB=cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,return_basis=True
    )

# gJ : Lande g-factor, p : parity(e parity)

E_A = np.unique(np.diag(H0_A))

dijq = pylcp.hamiltonians.XFmolecules.dipoleXandAstates(
    Xbasis, Abasis, UX=U_X
    )

hamiltonian = pylcp.hamiltonian(H0_X, H0_A, Bq_X, Bq_A, dijq,mass = mass)

In [11]:
connection = MongoClient("localhost:27017")
db = connection.db.Expected_model
max_parameters = db.distinct(key = 'params',filter={'target' : {'$gte' : 0.0666}})

In [12]:
start = time.time()

print(Expected_model_simulation.Expected_simulation(**max_parameters[0]))

print(time.time()-start)

100%|██████████| 5/5 [00:04<00:00,  1.12it/s]


0.0013148561476419495
73.31702542304993


In [5]:
pbounds = {'main_det':(10,20),'det_1':(70,170),'det_2':(5,30),'beta_1':(0,5),'beta_2':(0,15),'laseron':(138240,300000),'laseroff':(300001,600000)}

In [7]:
optimizer = BayesianOptimization(
    f = Expected_model_simulation.Expected_simulation,
    pbounds=pbounds,
    verbose=13,
    random_state=1,
)

In [8]:
from bayes_opt.util import load_logs

# New optimizer is loaded with previously seen points
# load_logs(optimizer, logs=["./logs_Exp_multi.json"])

FileNotFoundError: [Errno 2] No such file or directory: './logs_Exp_multi.json'

In [9]:
start = time.time()
optimizer.maximize(
    init_points=1,
    n_iter=0,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------


100%|██████████| 1/1 [00:03<00:00,  3.84s/it]


|  1        |  0.0      |  2.085    |  10.8     |  70.01    |  12.56    |  3.44e+05 |  1.532e+0 |  11.86    |
256.365740776062


In [60]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path='./logs_Exp_multi.json')
optimizer.subscribe(Events.OPTIMIZATION_STEP,logger)

In [61]:
start = time.time()
optimizer.maximize(
    init_points=5,
    n_iter=10,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  2        |  0.0      |  1.728    |  5.952    |  123.9    |  15.48    |  5.056e+0 |  1.713e+0 |  18.78    |
|  3        |  0.02726  |  0.1369   |  10.06    |  111.7    |  18.97    |  3.421e+0 |  1.703e+0 |  18.01    |
|  4        |  0.0      |  4.841    |  4.701    |  139.2    |  26.91    |  5.684e+0 |  1.52e+05 |  10.39    |
|  5        |  0.0      |  0.8492   |  13.17    |  79.83    |  15.53    |  5.874e+0 |  2.245e+0 |  16.92    |
|  6        |  0.0473   |  1.578    |  10.3     |  153.5    |  5.457    |  5.25e+05 |  2.982e+0 |  17.48    |
|  7        |  0.0      |  1.12     |  14.5     |  123.4    |  23.71    |  5.496e+0 |  1.88e+05 |  15.54    |
|  8        |  0.0      |  0.5403   |  9.576    |  141.3    |  18.9     |  3.72e+05 |  1.397e+0 |  12.15    |
|  9      

In [10]:
start = time.time()
optimizer.maximize(
    init_points=0,
    n_iter=30,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.0      |  2.646    |  7.78     |  155.1    |  14.86    |  3.28e+05 |  2.377e+0 |  10.4     |
|  2        |  0.0      |  3.826    |  1.643    |  122.2    |  24.21    |  5.061e+0 |  1.598e+0 |  12.02    |
|  3        |  0.01639  |  2.883    |  3.271    |  135.7    |  14.08    |  3.042e+0 |  2.007e+0 |  14.02    |
|  4        |  0.2334   |  1.615    |  10.69    |  118.0    |  5.67     |  5.048e+0 |  2.644e+0 |  19.74    |
|  5        |  0.0      |  2.207    |  7.635    |  128.4    |  25.99    |  3.008e+0 |  2.706e+0 |  11.11    |
|  6        |  0.02142  |  0.1765   |  10.53    |  151.0    |  16.91    |  3.743e+0 |  2.611e+0 |  10.2     |
|  7        |  0.1048   |  4.625    |  12.57    |  70.76    |  27.36    |  3.016e+0 |  2.205e+0 |  14.44    |
|  8      

C:\Users\qmopl\AppData\Local\Temp/ipykernel_10708/3589849535.py:83: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(total_func,v_trap_initial[0],v_trap_initial[-1],limit=100)[0]


|  29       |  0.2944   |  3.805    |  9.937    |  87.6     |  25.53    |  4.919e+0 |  1.623e+0 |  19.47    |
|  30       |  0.0      |  3.157    |  12.44    |  145.8    |  26.19    |  4.12e+05 |  2.233e+0 |  11.68    |
4919.5048615932465


In [15]:
optimizer.maximize(n_iter=1,init_points = 0)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  53       |  0.002721 |  4.385    |  0.326    |  116.6    |  15.66    |  4.285e+0 |  2.125e+0 |  18.62    |


In [11]:
optimizer.max

{'target': 0.2944409759829353,
 'params': {'beta_1': 3.8050995873529567,
  'beta_2': 9.936559576119455,
  'det_1': 87.59839010311943,
  'det_2': 25.530476380724924,
  'laseroff': 491912.16996315226,
  'laseron': 162300.30234814572,
  'main_det': 19.47056298409516}}